# tokenize pdfs

In [1]:
# This version outputs a CSV in normal form:
# slug, page, x, y, token
import pandas as pd
import pdfplumber
import csv
import json

In [ ]:
nopened = 0
nparsed = 0
nopenerror = 0
nparseerror = 0

def print_stats():
    global nopened, nparsed, nopenerror, nparseerror
    print('-----')
    print(f'Found {nopened} files, could not open {nopenerror}')
    print(f'Parsed {nparsed}, could not parse {nparseerror}')
    print('-----')


d = pd.read_csv('C:/Users/HP/Documents/work/DeepForm/deepform-master/files.csv')

f = open('C:/Users/HP/Documents/work/DeepForm/deepform-master/my-tokens.csv', mode='w')
csv = csv.writer(f)
csv.writerow(['slug','x0','y0','x1','y1','token'])

i=0

for index, row in d.iterrows():
#for i in range(3) :
    
    
    slug = row['pdf files']
    #slug = d.loc[i, "dc_slug"]
    fname = 'C:/Users/HP/Documents/work/DeepForm/deepform-master/pdfs/' + slug
    print('Extracting ' +  fname)

    try:
        pdf = pdfplumber.open(fname)
        nopened += 1
    except Exception as e:
        print(e)
        nopenerror += 1
        continue

    try:
        for p in range(len(pdf.pages)):
            for w in pdf.pages[p].extract_words():
                if '\0' not in w['text']:  # some tokens have nulls in them, which are not valid in a csv
                    csv.writerow([slug, 
                                p, 
                                float(w['x0']), 
                                float(w['top']), 
                                float(w['x1']), 
                                float(w['bottom']), 
                                w['text']])
        nparsed += 1
    except Exception as e:
        print(e)
        nparseerror +=1 

    #if (index % 100) == 0:
       # print_stats()

#i = i+1

print('-----')
print("Done!")
print(f'{len(d)} rows total')
print_stats()

# training the model

In [2]:
# Data extraction by deep learning, using a fully connected architecture over token windows.
# Engineered to extract total amounts, using a few custom features.
# Achieves up to 90% accuracy.
#
# jstray 2019-6-12

import keras as K
from keras.engine.input_layer import Input
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Lambda, concatenate
from keras.layers.embeddings import Embedding
from keras.backend import expand_dims, squeeze
import tensorflow as tf
import pandas as pd
import numpy as np
import csv
import re
import random
import pdfplumber
import os
import pickle
from decimal import Decimal

#import wandb
#from wandb.keras import WandbCallback

# uration
#run = init(project="jonathan_summer_1", entity="deepform", name="testing")

# = run.

In [3]:
# address some inteeface discrepancies when using tensorflow.keras
if "slice" not in K.__dict__ and K.backend == "tensorflow":
    # this is a good indicator that we are using tensorflow.keras

    try:
        # at first try to monkey patch what we need, will only work if keras-team keras is installed
        from keras import backend as KKK

        try:
            K.__dict__.update(
                is_tensor=KKK.is_tensor,
                slice=KKK.slice,
            )
        finally:
            del KKK
    except ImportError:
        # if that doesn't work we do a dirty copy of the code required
        import tensorflow as tf
        from tensorflow.python.framework import ops as tf_ops


        def is_tensor(x):
            return isinstance(x, tf_ops._TensorLike) or tf_ops.is_dense_tensor_like(x)


        def slice(x, start, size):
            x_shape = K.int_shape(x)
            if (x_shape is not None) and (x_shape[0] is not None):
                len_start = K.int_shape(start)[0] if is_tensor(start) else len(start)
                len_size = K.int_shape(size)[0] if is_tensor(size) else len(size)
                if not (len(K.int_shape(x)) == len_start == len_size):
                    raise ValueError('The dimension and the size of indices should match.')
            return tf.slice(x, start, size)

In [22]:
read_docs = 10 # how many docs to load, at most
window_len = 30 # size of token sequences to train on (and network size!)
vocab_size = 500
token_dims = 6 # number of features per token, including token hash
positive_fraction = 0.5
target_thresh = 0.8 # target match scores larger than this will becomes positive labels
epochs = 10
batch_size=1000
steps_per_epoch = 10
doc_acc_sample_size = 5 # how many documents to check extraction on after each epoch
penalize_missed = 5 # how much more a missed 1 counts than a missed 0 in output
val_split = 0.2
len_train = 80

source_data = 'C:/Users/HP/Documents/work/DeepForm/deepform-master/mytrain.csv'
pickle_destination = 'C:/Users/HP/Documents/work/DeepForm/deepform-master/source/save.p'

# ---- Load data and generate features ----

# Generator that reads raw training data
# For each document, yields an array of dictionaries, each of which is a token
def input_docs(max_docs=None):
    incsv = csv.DictReader(open(source_data, mode='r'))

    # Reconstruct documents by concatenating all rows with the same slug
    active_slug = None
    doc_rows = [] 
    num_docs = 0

    for row in incsv:	
        # throw out tokens that are too short, they won't help us
        token = row['token']
        if len(token) < 3:
            continue 

        if row['slug'] != active_slug:
            if active_slug:
                yield doc_rows
                num_docs += 1
                if max_docs and num_docs >= max_docs:
                    return
            doc_rows = [row]
            active_slug = row['slug']
        else:
            doc_rows.append(row)

    yield doc_rows


def is_required_amount(s):
    return re.search(r'\$?\d[\d,]+(\.\d\d)?',s) != None

def token_features(row, vocab_size):
    tokstr = row['token'].upper()
    return [ hash(tokstr) % vocab_size,
                     #float(row['page']), 
                     float(row['x0']),
                     float(row['y0']), 
                     float(len(tokstr)),
                     float(np.mean([c.isdigit() for c in tokstr])),
                     float(is_required_amount(tokstr)) ]

# Load raw training data, create our per-token features and binary labels
def load_training_data_nocache():
    slugs = []
    token_text = []
    features = []
    label1 = []
    for doc_tokens in input_docs(max_docs=read_docs):
        if len(doc_tokens) < window_len:
            continue # TODO pad shorter docs

        # Not training data, but used for evaluating results later
        slugs.append(doc_tokens[0]['slug']) # unique document ID, also PDF filename
        token_text.append([row['token'] for row in doc_tokens])

        features.append([token_features(row, vocab_size) for row in doc_tokens])

        # threshold fuzzy matching score with our target field, to get binary labels 
        label1.append([(0 if float(row['Wages,_tips_and_other_compensation']) < target_thresh else 1) for row in doc_tokens])
        
    print("Length of slugs in load_training_data_nocache = ", len(slugs))
    return slugs, token_text, features, label1

# Because generating the list of features is so expensive, we cache it on disk

def load_training_data():
    if os.path.isfile(pickle_destination):
        print('Loading training data from cache...')
        #slugs, token_text, features = pickle.load(open(pickle_destination, 'rb'))
        slugs, token_text, features, label1 = pickle.load(open(pickle_destination, 'rb'))
    else:
        print('Loading training data...')
        #slugs, token_text, features = load_training_data_nocache()
        slugs, token_text, features, label1 = load_training_data_nocache()
        
        print('Saving training data to cache...')
        #pickle.dump((slugs, token_text, features), open(pickle_destination, 'wb'))
        pickle.dump((slugs, token_text, features, label1), open(pickle_destination, 'wb'))

    # Trim the training data so we can sweep across various training data sizes
    print("Length of slugs in load_training_data before modification = ", len(slugs))
    slugs = slugs[:len_train]
    print("Length of slugs in load_training_data after modification = ", len(slugs))
    token_text = token_text[:len_train]
    features = features[:len_train]
    label1 = label1[:len_train]

    return slugs, token_text, features, label1

In [23]:
# ---- Resample features,labels as windows ----

# returns a window of tokens, labels at a random position in a random document
def one_window_unbalanced(features, labels, window_len):
    #def one_window_unbalanced(features, window_len, ):
    '''if(len(features)<2):
        doc_idx = 0
        return None, None
    else:'''    
    doc_idx = random.randint(0,len(features)-1)
    doc_len = len(features[doc_idx])
    tok_idx = random.randint(0, doc_len-window_len)
    return features[doc_idx][tok_idx : tok_idx+window_len], label1[doc_idx][tok_idx : tok_idx+window_len]

# control the fraction of windows that include a positive label. not efficient.
def one_window(features, label1, window_len, positive_fraction):
    f,l = one_window_unbalanced(features, label1, window_len)   
    if random.random() > positive_fraction: # mostly positive examples
        while not 1 in l:
            if(len(features)>1):
                f,l = one_window_unbalanced(features, label1, window_len)
            else:
                continue
    return f,l


def windowed_generator(features, labels):
    #def windowed_generator(features):    
    # Create empty arrays to contain batch of features and labels#
    
    batch_features = np.zeros((batch_size, window_len, token_dims))
    batch_labels = np.zeros((batch_size, window_len))

    while True:
        for i in range(batch_size):
            features1,labels1 = one_window(features, label1, window_len, positive_fraction)
            #features1 = one_window_unbalanced(features, window_len)
            if len(features1) == 6:
                batch_features[i,:,:] = features1
            if(len(labels1) == 30):    
                batch_labels[i,:] = labels1
                
        yield batch_features, batch_labels

In [25]:
# ---- Custom loss function is basically MSE but high penalty for missing a 1 label ---

def missed_token_loss(one_penalty):
   
    def _missed_token_loss(y_true, y_pred):
        expected_zero = tf.cast(tf.math.equal(y_true,0), tf.float32)
        s = y_pred*expected_zero
        zero_loss = K.backend.mean(K.backend.square(s))
        expected_one = tf.cast(tf.math.equal(y_true,1), tf.float32)
        t = one_penalty*(1-y_pred)*expected_one
        one_loss = K.backend.mean(K.backend.square(t))
        return zero_loss + one_loss

    return _missed_token_loss # closes over one_penalty

# --- Specify network ---

def create_model():
    indata = Input((window_len, token_dims))

    # split into the hash and the rest of the token features, embed hash as one-hot, then merge
    tok_hash = Lambda( lambda x: squeeze(tf.slice(x, (0,0,0), (-1,-1,1)),axis=2))(indata)
    tok_features = Lambda( lambda x: tf.slice(x, (0,0,1), (-1,-1,-1)))(indata)
    embed = Embedding(vocab_size, 32)(tok_hash)
    merged = concatenate([embed, tok_features], axis=2)

    f = Flatten()(merged)
    d1 = Dense(window_len*token_dims*5, activation='sigmoid')(f)
    d2 = Dropout(0.3)(d1)
    d3 = Dense(window_len*token_dims, activation='sigmoid')(d2)
    d4 = Dropout(0.3)(d3)
    d5 = Dense(window_len, activation='elu')(d4)

    model = Model(inputs=[indata], outputs=[d5])
    model.compile(
        optimizer='adam', 
        loss=missed_token_loss(penalize_missed), 
        metrics=['acc'])

    return model

# --- Predict ---
# Our network is windowed, so we have to aggregate windows to get a final score

# Returns vector of token scores
def predict_scores(model, features, window_len):
    doc_len = len(features)
    num_windows = doc_len-window_len

    windowed_features = np.array([features[i:i+window_len] for i in range(num_windows)])
    window_scores = model.predict(windowed_features)

    scores = np.zeros(doc_len)
    for i in range(num_windows):
        scores[i:i+window_len] += window_scores[i] # would max work better than sum?
    return scores

# returns text, score of best answer
def predict_answer(model, features, token_text, window_len):
	scores = predict_scores(model, features, window_len)
	best_score_idx = np.argmax(scores)
	best_score_text = token_text[best_score_idx]
	return best_score_text, scores[best_score_idx]


# returns text of correct answer,
def correct_answer(features, label1, token_text):
	answer_idx = np.argmax(label1)
	answer_text = token_text[answer_idx]
	return answer_text


# Calculate accuracy of answer extraction over num_to_test docs, print diagnostics while we do so
def compute_accuracy(model, window_len, slugs, token_text, features, label1, num_to_test):
	acc = 0.0
	for i in range(num_to_test):
		doc_idx = random.randint(0, len(slugs)-1)
		predict_text, predict_score = predict_answer(model, features[doc_idx], token_text[doc_idx], window_len)
		answer_text = correct_answer(features[doc_idx], label1[doc_idx], token_text[doc_idx])
		print(f'{slugs[doc_idx]}: guessed "{predict_text}" with score {predict_score}, correct "{answer_text}"')
		if predict_text==answer_text:
			acc+=1
	return acc/num_to_test


# ---- Custom callback to log document-level accuracy ----

class DocAccCallback(K.callbacks.Callback):
	def __init__(self, window_len, slugs, token_text, features, label1, num_to_test, logname):
		self.window_len = window_len
		self.slugs = slugs
		self.token_text = token_text
		self.features = features
		self.label1 = label1
		self.num_to_test = num_to_test
		self.logname = logname

	def on_epoch_end(self, epoch, logs):
		acc = compute_accuracy(self.model,
			self.window_len,
			self.slugs,
			self.token_text,
			self.features,
			self.label1,
			self.num_to_test+epoch)  # test more docs later in training, for more precise acc
		print(f'This epoch {self.logname}: {acc}')
		#wandb.log({self.logname:acc})


# --- MAIN ----

#print('uration:')
#print()
maxarray = []
summation = 0
max_length = 0
slugs, token_text, features, label1 = load_training_data()
print(f'Loaded {len(features)}')
#max_length = max([len(x) for x in label1])

for x in label1:
    maxarray.append(len(x))
    summation = summation + len(x)
    
if (maxarray):    
    max_length = max(maxarray)
    print(f'Max document size {max_length}')
    
    avg_length = summation/len(label1)
    print(f'Average document size {avg_length}')

else:
    print("labels don't exist here")
    
# split into train and test
slugs_train = []
token_text_train = []
features_train = []
labels_train = []
slugs_val = []
token_text_val = []
features_val = []
labels_val = []
for i in range(len(features)):
	if random.random() < val_split:
		slugs_val.append(slugs[i])
		token_text_val.append(token_text[i])
		features_val.append(features[i])
		labels_val.append(label1[i])
	else:
		slugs_train.append(slugs[i])
		token_text_train.append(token_text[i])		
		features_train.append(features[i])
		labels_train.append(label1[i])

print(f'Training on {len(features_train)}, validating on {len(features_val)}')

model = create_model()
print(model.summary())


model.fit_generator(
    windowed_generator(features_train, labels_train),
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    callbacks=[ 
        #WandbCallback(),
        DocAccCallback(	window_len, 
                                        slugs_train, 
                                        token_text_train, 
                                        features_train, 
                                        labels_train, 
                                        doc_acc_sample_size,
                                        'doc_train_acc'),
        DocAccCallback(	window_len, 
                                        slugs_val, 
                                        token_text_val, 
                                        features_val, 
                                        labels_val, 
                                        doc_acc_sample_size,
                                        'doc_val_acc')
    ])

Loading training data from cache...
Length of slugs in load_training_data before modification =  10
Length of slugs in load_training_data after modification =  10
Loaded 10
Max document size 1968
Average document size 932.2
Training on 8, validating on 2
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 30, 6)]      0                                            
__________________________________________________________________________________________________
lambda_14 (Lambda)              (None, 30)           0           input_8[0][0]                    
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 30, 32)       16000       lambda_14[0][0]                  
___________________________________

CAPRUS W2.pdf: guessed "HILLSBOROOR97124" with score 14.095292538404465, correct "131558.7"
This epoch doc_val_acc: 0.0
Epoch 5/10
10/10 [==============================] - 1s 84ms/step - loss: 0.4872 - acc: 0.0249
Intel W2.pdf: guessed "1756.96" with score 14.345356047153473, correct "106468.39"
VMWARE W2.pdf: guessed "AUSTIN" with score 14.437369972467422, correct "202676.58"
Intel W2.pdf: guessed "1756.96" with score 14.345356047153473, correct "106468.39"
Intel W2.pdf: guessed "1756.96" with score 14.345356047153473, correct "106468.39"
OpenLogix W2.pdf: guessed "1Wages,tips,othercompe7ns4at1io4n0.00" with score 14.419835925102234, correct "NoticetoEmployee"
ISN W2.pdf: guessed "Other" with score 14.46688911318779, correct "135405.59"
Oath W2.pdf: guessed "tax" with score 14.49444654583931, correct "150207.24"
OpenLogix W2.pdf: guessed "1Wages,tips,othercompe7ns4at1io4n0.00" with score 14.419835925102234, correct "NoticetoEmployee"
Oath W2.pdf: guessed "tax" with score 14.4944465458

Sonus Software W2.pdf: guessed "Employeeï¿½sSSAnumber" with score 15.640243798494339, correct "2019"
Sonus Software W2.pdf: guessed "Employeeï¿½sSSAnumber" with score 15.640243798494339, correct "2019"
Sonus Software W2.pdf: guessed "Employeeï¿½sSSAnumber" with score 15.640243798494339, correct "2019"
CAPRUS W2.pdf: guessed "HILLSBOROOR97124" with score 15.556104272603989, correct "131558.7"
This epoch doc_val_acc: 0.0
Epoch 9/10
10/10 [==============================] - 1s 77ms/step - loss: 0.4657 - acc: 0.0259
Intel W2.pdf: guessed "Mission" with score 15.321691662073135, correct "106468.39"
ISN W2.pdf: guessed "Other" with score 15.48041957616806, correct "135405.59"
Stategic Systems W2.pdf: guessed "1Wages,tips,othercompe9ns6at8io6n1.89" with score 15.406005561351776, correct "96861.89"
Intel W2.pdf: guessed "Mission" with score 15.321691662073135, correct "106468.39"
OpenLogix W2.pdf: guessed "66.65" with score 15.379662841558456, correct "NoticetoEmployee"
Qual W2FORM.pdf: guessed